In [ ]:
import pickle as pkl
import pandas as pd
import aegnn
import torch
import torch_geometric

import argparse
import logging
import os
import numpy as np
import pandas as pd
import pytorch_lightning.metrics.functional as pl_metrics

from torch_geometric.data import Batch
from torch_geometric.utils import subgraph
from typing import Iterable, Tuple

from torch_geometric.data import Data
from tqdm import tqdm
from typing import List
from torch.nn import Linear
from torch.nn.functional import elu
from torch_geometric.nn.conv import SplineConv
from torch_geometric.nn.norm import BatchNorm
from torch_geometric.transforms import Cartesian

: 

For 4000 samples, wrt their own events/nodes:
```
acc_Mflop_per_ev = 134.74224981457488, avg_Mflop_per_ev = 0.03368556245364372
dense: std acc_Mflop_per_ev = 1170.527851753108, std avg_Mflop_per_ev = 0.29263196293827703
```

For 100 samples, wrt their own events/nodes:
```
acc_Mflop_per_ev = 3.414027443525552, avg_Mflop_per_ev = 0.034140274435255524
dense: std acc_Mflop_per_ev = 28.595094207357473, std avg_Mflop_per_ev = 0.28595094207357474
```

Corrected formula for calculating FLOPS (according to updated paper), leading to:

For 4000 samples, wrt their own events/nodes:
```
acc_Mflop_per_ev = 240.93893365788432, avg_Mflop_per_ev = 0.06023473341447108
dense: std acc_Mflop_per_ev = 2096.876591199398, std avg_Mflop_per_ev = 0.5242191477998495
```

In [ ]:
data_module = aegnn.datasets.NCars(batch_size=1, shuffle=False)
data_module.setup()
dm = data_module.train_dataset

# cnt = 0
# for i, dms in enumerate(dm):
#     cnt += 1
#     print(i)

# print(cnt)

# num_trials=100
# nodes=[]
# nodes_cnt=0
# for index in tqdm(range(num_trials)):
#     sample = dm[index % len(dm)]
#     nodes.append(sample.num_nodes)
#     nodes_cnt += sample.num_nodes
# print(nodes_cnt)

print(dm.dims)

: 

if add async code into train.py
```
File "scripts/train.py", line 97, in <module>
    main(arguments)
  File "scripts/train.py", line 91, in main
    trainer.fit(model, datamodule=dm)
  File "/users/yyang22/anaconda3/envs/aegnn/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py", line 552, in fit
    self._run(model)
  File "/users/yyang22/anaconda3/envs/aegnn/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py", line 922, in _run
    self._dispatch()
  File "/users/yyang22/anaconda3/envs/aegnn/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py", line 990, in _dispatch
    self.accelerator.start_training(self)
  File "/users/yyang22/anaconda3/envs/aegnn/lib/python3.8/site-packages/pytorch_lightning/accelerators/accelerator.py", line 92, in start_training
    self.training_type_plugin.start_training(trainer)
  File "/users/yyang22/anaconda3/envs/aegnn/lib/python3.8/site-packages/pytorch_lightning/plugins/training_type/training_type_plugin.py", line 161, in 
start_training
    self._results = trainer.run_stage()
  File "/users/yyang22/anaconda3/envs/aegnn/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py", line 1000, in run_stage
    return self._run_train()
  File "/users/yyang22/anaconda3/envs/aegnn/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py", line 1049, in _run_train
    self.fit_loop.run()
  File "/users/yyang22/anaconda3/envs/aegnn/lib/python3.8/site-packages/pytorch_lightning/loops/base.py", line 111, in run
    self.advance(*args, **kwargs)
  File "/users/yyang22/anaconda3/envs/aegnn/lib/python3.8/site-packages/pytorch_lightning/loops/fit_loop.py", line 200, in advance
    epoch_output = self.epoch_loop.run(train_dataloader)
  File "/users/yyang22/anaconda3/envs/aegnn/lib/python3.8/site-packages/pytorch_lightning/loops/base.py", line 118, in run
    output = self.on_run_end()
  File "/users/yyang22/anaconda3/envs/aegnn/lib/python3.8/site-packages/pytorch_lightning/loops/epoch/training_epoch_loop.py", line 235, in on_run_end
    self._on_train_epoch_end_hook(processed_outputs)
  File "/users/yyang22/anaconda3/envs/aegnn/lib/python3.8/site-packages/pytorch_lightning/loops/epoch/training_epoch_loop.py", line 276, in _on_train_epoch_end_hook
    trainer_hook(processed_epoch_output)
  File "/users/yyang22/anaconda3/envs/aegnn/lib/python3.8/site-packages/pytorch_lightning/trainer/callback_hook.py", line 109, in on_train_epoch_end
    callback.on_train_epoch_end(self, self.lightning_module)
  File "/users/yyang22/anaconda3/envs/aegnn/lib/python3.8/site-packages/pytorch_lightning/callbacks/model_checkpoint.py", line 310, in on_train_epoch_end  
    self.save_checkpoint(trainer)
  File "/users/yyang22/anaconda3/envs/aegnn/lib/python3.8/site-packages/pytorch_lightning/callbacks/model_checkpoint.py", line 386, in save_checkpoint
    self._save_none_monitor_checkpoint(trainer, monitor_candidates)
  File "/users/yyang22/anaconda3/envs/aegnn/lib/python3.8/site-packages/pytorch_lightning/callbacks/model_checkpoint.py", line 742, in _save_none_monitor_checkpoint
    self._save_model(trainer, filepath)
  File "/users/yyang22/thesis/aegnn_project/aegnn/aegnn/utils/callbacks/checkpoint_full_model.py", line 14, in _save_model
    torch.save(trainer.model, filepath)
  File "/users/yyang22/anaconda3/envs/aegnn/lib/python3.8/site-packages/torch/serialization.py", line 380, in save
    _save(obj, opened_zipfile, pickle_module, pickle_protocol)
  File "/users/yyang22/anaconda3/envs/aegnn/lib/python3.8/site-packages/torch/serialization.py", line 589, in _save
    pickler.dump(obj)
AttributeError: Can't pickle local object 'make_model_asynchronous.<locals>.async_forward'
```

if I use wandb:
```
File "scripts/train.py", line 97, in <module>
    main(arguments)
  File "scripts/train.py", line 91, in main
    trainer.fit(model, datamodule=dm)
  File "/users/yyang22/anaconda3/envs/aegnn/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py", line 552, in fit
    self._run(model)
  File "/users/yyang22/anaconda3/envs/aegnn/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py", line 922, in _run
    self._dispatch()
  File "/users/yyang22/anaconda3/envs/aegnn/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py", line 990, in _dispatch
    self.accelerator.start_training(self)
  File "/users/yyang22/anaconda3/envs/aegnn/lib/python3.8/site-packages/pytorch_lightning/accelerators/accelerator.py", line 92, in start_training
    self.training_type_plugin.start_training(trainer)  
  File "/users/yyang22/anaconda3/envs/aegnn/lib/python3.8/site-packages/pytorch_lightning/plugins/training_type/training_type_plugin.py", line 161, in 
start_training
    self._results = trainer.run_stage()
  File "/users/yyang22/anaconda3/envs/aegnn/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py", line 1000, in run_stage
    return self._run_train()
  File "/users/yyang22/anaconda3/envs/aegnn/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py", line 1049, in _run_train
    self.fit_loop.run()
  File "/users/yyang22/anaconda3/envs/aegnn/lib/python3.8/site-packages/pytorch_lightning/loops/base.py", line 111, in run
    self.advance(*args, **kwargs)
File "/users/yyang22/anaconda3/envs/aegnn/lib/python3.8/site-packages/pytorch_lightning/loops/fit_loop.py", line 200, in advance
    epoch_output = self.epoch_loop.run(train_dataloader)
  File "/users/yyang22/anaconda3/envs/aegnn/lib/python3.8/site-packages/pytorch_lightning/loops/base.py", line 118, in run
    output = self.on_run_end()
  File "/users/yyang22/anaconda3/envs/aegnn/lib/python3.8/site-packages/pytorch_lightning/loops/epoch/training_epoch_loop.py", line 235, in on_run_end
    self._on_train_epoch_end_hook(processed_outputs)   
  File "/users/yyang22/anaconda3/envs/aegnn/lib/python3.8/site-packages/pytorch_lightning/loops/epoch/training_epoch_loop.py", line 276, in _on_train_e
poch_end_hook
    trainer_hook(processed_epoch_output)
  File "/users/yyang22/anaconda3/envs/aegnn/lib/python3.8/site-packages/pytorch_lightning/trainer/callback_hook.py", line 109, in on_train_epoch_end
    callback.on_train_epoch_end(self, self.lightning_module)
  File "/users/yyang22/anaconda3/envs/aegnn/lib/python3.8/site-packages/pytorch_lightning/callbacks/model_checkpoint.py", line 310, in on_train_epoch_end
    self.save_checkpoint(trainer)
  File "/users/yyang22/anaconda3/envs/aegnn/lib/python3.8/site-packages/pytorch_lightning/callbacks/model_checkpoint.py", line 386, in save_checkpoint
    self._save_none_monitor_checkpoint(trainer, monitor_candidates)
  File "/users/yyang22/anaconda3/envs/aegnn/lib/python3.8/site-packages/pytorch_lightning/callbacks/model_checkpoint.py", line 742, in _save_none_monitor_checkpoint
    self._save_model(trainer, filepath)
  File "/users/yyang22/thesis/aegnn_project/aegnn/aegnn/utils/callbacks/checkpoint_full_model.py", line 14, in _save_model
    torch.save(trainer.model, filepath)
File "/users/yyang22/anaconda3/envs/aegnn/lib/python3.8/site-packages/torch/serialization.py", line 380, in save
    _save(obj, opened_zipfile, pickle_module, pickle_protocol)
  File "/users/yyang22/anaconda3/envs/aegnn/lib/python3.8/site-packages/torch/serialization.py", line 589, in _save
    pickler.dump(obj)
AttributeError: Can't pickle local object 'Settings._validator_factory.<locals>.helper'
```
Problems seem to be at logger function

In [ ]:
import wandb
from wandb import sdk as wandb_sdk

wandb.init(project="aegnn", entity="yyfteam")
# log_settings = wandb.Settings(start_method="thread")
log_settings = wandb.Settings(start_method="fork")
wandb_sdk.Settings()

In [2]:
import aegnn
import torch
import torch_geometric

edge_attr = torch_geometric.transforms.Cartesian(cat=False, max_value=10.0)

print(type(edge_attr))

<class 'torch_geometric.transforms.cartesian.Cartesian'>


In [10]:
import torch
import torch.nn as nn
from torch.optim.lr_scheduler import LambdaLR

initial_lr = 0.5



class model(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=3, kernel_size=3)

    def forward(self, x):
        pass

net_1 = model()

def LRPolicy(epoch):
    if epoch < 2:
        return 1
    else:
        return 0.1

optimizer_1 = torch.optim.Adam(net_1.parameters(), lr = initial_lr)
scheduler_1 = LambdaLR(optimizer_1, lr_lambda=LRPolicy)

print("init lr", optimizer_1.defaults['lr'])

for epoch in range(1, 11):
    # train
    optimizer_1.zero_grad()
    optimizer_1.step()
    print("lr of %dth epoch: %f" % (epoch, optimizer_1.param_groups[0]['lr']))
    scheduler_1.step()

init lr 0.5
lr of 1th epoch: 0.500000
lr of 2th epoch: 0.500000
lr of 3th epoch: 0.050000
lr of 4th epoch: 0.050000
lr of 5th epoch: 0.050000
lr of 6th epoch: 0.050000
lr of 7th epoch: 0.050000
lr of 8th epoch: 0.050000
lr of 9th epoch: 0.050000
lr of 10th epoch: 0.050000


In [25]:
import os
import shutil
import glob
import sys

path='../aegnn_results/training_results/latest'
if not os.path.exists(path):
    os.makedirs(path)
else:
    # clean
    try:
        shutil.rmtree(path)
    except OSError as e:
        print("Error: %s : %s" % (path, e.strerror))
    
    # rebuild
    os.makedirs(path)

src_model = sorted(glob.glob(r'/space/yyang22/datasets/data/scratch/checkpoints/ncars/recognition/*/*.pt'), key=os.path.getctime)[-1]
dst_model = os.path.join(path,'latest_model.pt')

src_log = sorted(glob.glob(r'/space/yyang22/datasets/data/scratch/debug/*'), key=os.path.getctime)[-1]
dst_log = os.path.join(path,'latest.log')

print(src_model,dst_model,src_log,dst_log)
try:
    shutil.copy2(src_model, dst_model)
except IOError as e:
    print("Unable to copy file. %s" % e)
except:
    print("Unexpected error:", sys.exc_info())

try:
    shutil.copy2(src_log, dst_log)
except IOError as e:
    print("Unable to copy file. %s" % e)
except:
    print("Unexpected error:", sys.exc_info())





/space/yyang22/datasets/data/scratch/checkpoints/ncars/recognition/20221121142631/epoch=99-step=81099.pt ../aegnn_results/training_results/latest/latest_model.pt /space/yyang22/datasets/data/scratch/debug/20221121-142631.log ../aegnn_results/training_results/latest/latest.log


In [29]:
import os

cmd = 'python3 ../../test_bkgnd.py'
os.system(cmd)

test is started:


  8%|▊         | 8/100 [00:24<04:36,  3.00s/it]

In [ ]:
CUDA_VISIBLE_DEVICES=5 wandb agent --count 4 yyfteam/aegnn/yzqyfzg6